In [9]:
import os
import numpy as np
import pandas as pd
import pickle as pkl
import imageio
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from PIL import Image
import PIL
from astropy.nddata.utils import Cutout2D
from scipy import ndimage
import time
import matplotlib.pyplot as plt

In [2]:
datadir = os.path.join('D:\\','OneDrive - Northwestern University','Photos', 'bubbleimages')

In [3]:
# define functions to normalize image
def normalize(arr):
    arr = np.array(arr)
    arr = arr - np.min(arr)
    arr = arr / np.max(arr)
    return np.array(arr)

def gaussian_normalize(arr):
    arr = np.array(arr)
    arr = arr - np.mean(arr)
    arr = arr / np.std(arr)
    return np.array(arr)

In [4]:
df = pkl.load(open("FeaturesDataFrame.p", "rb"))
y = np.array(df['bubblecount'])
x = np.array(df[['edgefeature', 'blobfeature', 'blobpeakfeature']])
for i in range(3):
    x[:,i] = gaussian_normalize(x[:,i])
print(x.shape, y.shape)

(2783, 3) (2783,)


In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(2226, 3) (557, 3) (2226,) (557,)


In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(3,)),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5)
])

In [7]:
model.compile(optimizer = 'adam', 
               loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
               metrics = ["accuracy", tf.keras.metrics.CategoricalAccuracy()])
model.fit(x_train,y_train,epochs = 10)
test_accuracy = model.evaluate(x_test, y_test, verbose = 3)
print("\nLoss: %.4f, Accuracy: %.4f, Categorical Accuracy: %.4f"%tuple(test_accuracy))

Epoch 1/10
2226/2226 [==============================] - 0s 214us/sample - loss: 0.7334 - acc: 0.8261 - categorical_accuracy: 0.7412
Epoch 2/10
2226/2226 [==============================] - 0s 79us/sample - loss: 0.3436 - acc: 0.8985 - categorical_accuracy: 0.7251
Epoch 3/10
2226/2226 [==============================] - 0s 94us/sample - loss: 0.2892 - acc: 0.9097 - categorical_accuracy: 0.7161
Epoch 4/10
2226/2226 [==============================] - 0s 45us/sample - loss: 0.2737 - acc: 0.9173 - categorical_accuracy: 0.7273
Epoch 5/10
2226/2226 [==============================] - 0s 90us/sample - loss: 0.2742 - acc: 0.9155 - categorical_accuracy: 0.7192
Epoch 6/10
2226/2226 [==============================] - 0s 85us/sample - loss: 0.2619 - acc: 0.9142 - categorical_accuracy: 0.7192
Epoch 7/10
2226/2226 [==============================] - 0s 100us/sample - loss: 0.2647 - acc: 0.9151 - categorical_accuracy: 0.7174
Epoch 8/10
2226/2226 [==============================] - 0s 56us/sample - loss: 0.

In [8]:
y_pred = model.predict_classes(x_test)
print(confusion_matrix(y_test,y_pred))

[[377  26   0]
 [ 28 123   0]
 [  0   3   0]]
